<a href="https://colab.research.google.com/github/acmilannesta/Bert-embedding/blob/master/GoogleQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Dec 10 14:44:16 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.36       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    35W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!pip install keras-bert
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import os, gc
import codecs
from keras.layers import *
from keras.callbacks import Callback
from keras.models import Model, load_model
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, GroupKFold, KFold
from scipy.stats import spearmanr
from tqdm import tqdm
from keras_bert import load_trained_model_from_checkpoint, Tokenizer, AdamWarmup, calc_train_steps, get_custom_objects
from hyperopt import fmin, hp, tpe, STATUS_OK, Trials


Using TensorFlow backend.


In [0]:
data_path = 'drive/My Drive/GoogleQA/Data/'
train = pd.read_csv(data_path+'train.csv')
test = pd.read_csv(data_path+'test.csv')
sub = pd.read_csv(data_path+'sample_submission.csv')

# train_q = pd.read_csv(data_path+'train_q.csv')
# test_q = pd.read_csv(data_path+'test_q.csv')
# train_a = pd.read_csv(data_path+'train_answer.csv')
# test_a = pd.read_csv(data_path+'test_answer.csv')

# df_aux = pd.concat([train[['host', 'category']], test[['host', 'category']]], 0)
# aux1 = pd.get_dummies(df_aux['category'], drop_first=True)
# aux2 = pd.get_dummies(df_aux['host'].apply(lambda x: x.split('.')[0]), drop_first=True)
# aux = pd.concat([aux1, aux2], 1)
# train_aux = pd.concat([train, aux.iloc[:len(train), ]], 1)
# test_aux =  pd.concat([test, aux.iloc[len(train):, ]], 1)

In [0]:
test.columns

Index(['qa_id', 'question_title', 'question_body', 'question_user_name',
       'question_user_page', 'answer', 'answer_user_name', 'answer_user_page',
       'url', 'category', 'host'],
      dtype='object')

##Chunknize answer

In [0]:
# def answer_chunk(df):
#     answer_col = ['qa_id'] + [col for col in df.columns if col.startswith('answer') and
#                 col not in ['answer_user_name', 'answer_user_page']] 
#     answer = df[answer_col]

#     answer_chunk = pd.DataFrame()
#     for row in tqdm(range(len(answer))):
#         if len(answer.loc[row, 'answer']) % 256 != 0:
#             replicate_row = len(answer.loc[row, 'answer']) // 256 + 1
#         else:
#             replicate_row = len(answer.loc[row, 'answer']) // 256
#         for i in range(replicate_row):
#             tmp = answer.loc[row].copy()
#             tmp.loc['answer_chunk'] = tmp['answer'][i*256: (i+1)*256]
#             answer_chunk = answer_chunk.append(tmp, ignore_index=True)
#     return answer_chunk

# def question_chunk(df):
#     q_col = ['qa_id'] + [col for col in df.columns if col.startswith('question') and
#                 col not in ['question_user_name', 'question_user_name']] 
#     q = df[q_col]

#     q_chunk = pd.DataFrame()
#     for row in tqdm(range(len(q))):
#         if len(q.loc[row, 'question_body']) % 256 != 0:
#             replicate_row = len(q.loc[row, 'question_body']) // 256 + 1
#         else:
#             replicate_row = len(q.loc[row, 'question_body']) // 256
#         for i in range(replicate_row):
#             tmp = q.loc[row].copy()
#             tmp.loc['qbody_chunk'] = tmp['question_body'][i*256: (i+1)*256]
#             q_chunk = q_chunk.append(tmp, ignore_index=True)
#     return q_chunk

# train_q, test_q = question_chunk(train), question_chunk(test)
# train_a, test_a = answer_chunk(train), answer_chunk(test)

100%|██████████| 476/476 [00:09<00:00, 52.14it/s]


In [8]:
MAXLEN = 512 #@param {type:"slider", min:128, max:512, step:32}
BATCH_SIZE = 4 #@param {type:'slider', min:4, max:32, step:4}
NUM_EPOCHS = 3
NUM_CLASSES = 30
LR = 5e-5
MIN_LR = 0
# OUTPUT_TRAIN = 'train_bert_ipredcv1415_oof.csv'
# OUTPUT_TEST = 'test_bert_large.npy'
model_path = 'uncased_L-12_H-768_A-12' #@param ['uncased_L-12_H-768_A-12', 'wwm_uncased_L-24_H-1024_A-16', 'uncased_L-24_H-1024_A-16']
# target_q_col = train.columns.tolist()[11:32]
# target_a_col = train.columns.tolist()[32:42]
target_col = train.columns.tolist()[11:42]

['question_asker_intent_understanding',
 'question_body_critical',
 'question_conversational',
 'question_expect_short_answer',
 'question_fact_seeking',
 'question_has_commonly_accepted_answer',
 'question_interestingness_others',
 'question_interestingness_self',
 'question_multi_intent',
 'question_not_really_a_question',
 'question_opinion_seeking',
 'question_type_choice',
 'question_type_compare',
 'question_type_consequence',
 'question_type_definition',
 'question_type_entity',
 'question_type_instructions',
 'question_type_procedure',
 'question_type_reason_explanation',
 'question_type_spelling',
 'question_well_written',
 'answer_helpful',
 'answer_level_of_information',
 'answer_plausible',
 'answer_relevance',
 'answer_satisfaction',
 'answer_type_instructions',
 'answer_type_procedure',
 'answer_type_reason_explanation',
 'answer_well_written']

In [0]:
token_dict = {}
with codecs.open(os.path.join(model_path, 'vocab.txt'), 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)
tokenizer = Tokenizer(token_dict)

In [0]:
def convert_data(data_df, branch='training'):
    data_df = data_df.reset_index(drop=True)
    global tokenizer
    global MAXLEN
    global target_col
    # global target_a_col
    # global target_q_col
    q_title, q_body, answer = [], [], []
    for i in tqdm(range(len(data_df))):
        answer_ids, _ = tokenizer.encode(data_df.loc[i, 'answer'][:MAXLEN])
        answer.append(answer_ids)                 
        q_title_ids, _ = tokenizer.encode(data_df.loc[i, 'question_title'][:MAXLEN])
        q_title.append(q_title_ids)
        q_body_ids, _ = tokenizer.encode(data_df.loc[i, 'question_body'][:MAXLEN])
        q_body.append(q_body_ids)
    if branch == 'training':
        targets = data_df[target_col]
        return [q_title, q_body, answer], np.array(targets)
    else:
        return [q_title, q_body, answer]



In [0]:
"""## Data Generator"""
def seq_padding(X, padding=0):
    L = [len(x) for x in X]
    ML = max(L)
    return np.array([np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x for x in X])

class data_generator:
    def __init__(self, data, batch_size=BATCH_SIZE, branch='train'):
        self.data = data
        self.batch_size = batch_size
        self.branch = branch
        # self.q_a = q_a
        self.steps = len(self.data) // self.batch_size
        if len(self.data) % self.batch_size != 0:
            self.steps += 1

    def __len__(self):
        return self.steps

    def __iter__(self):
        while True:
            if self.branch == 'train':
                np.random.shuffle(self.data)
            for i in range(self.steps):
                d = self.data[i * self.batch_size: (i + 1) * self.batch_size]
                X1 = seq_padding([x[0] for x in d])
                X2 = np.zeros_like(X1)
                X3 = seq_padding([x[1] for x in d])
                X4 = np.zeros_like(X3)
                X5 = seq_padding([x[2] for x in d])
                X6 = np.zeros_like(X5)
                if self.branch == 'test':
                    # aux = np.array([x[3] for x in d])
                    yield [X1, X2, X3, X4, X5, X6]
                else:
                    Y = np.array([x[3] for x in d])
                    # aux = np.array([x[4] for x in d])
                    yield [X1, X2, X3, X4, X5, X6], Y



In [0]:
def model_build(len_train):
    global NUM_CLASSES
    global BATCH_SIZE
    global NUM_EPOCHS
    global MIN_LR
    global LR
    global MAXLEN

    bert_model = load_trained_model_from_checkpoint(
        os.path.join(model_path, 'bert_config.json'),
        os.path.join(model_path, 'bert_model.ckpt'),
        seq_len = MAXLEN,
        trainable=True,
    )


    q_in = Input(shape=(None,))
    q2_in = Input(shape=(None,))
    qb_in = Input(shape=(None,))
    qb2_in = Input(shape=(None,))

    a_in = Input(shape=(None,))
    a2_in = Input(shape=(None,))
    # aux_in = Input(shape=(aux.shape[1], ))

    a_inputs = bert_model([a_in, a2_in])
    a_outputs = Lambda(lambda x: x[:, 0])(a_inputs)

    q_inputs = bert_model([q_in, q2_in])
    q_outputs = Lambda(lambda x: x[:, 0])(q_inputs)
    qb_inputs = bert_model([qb_in, qb2_in])
    qb_outputs = Lambda(lambda x: x[:, 0])(qb_inputs)

    dense = concatenate([q_outputs, qb_outputs, a_outputs])
    outputs = Dense(NUM_CLASSES, activation='softmax')(dense)
    model = Model([q_in, q2_in, qb_in, qb2_in, a_in, a2_in], outputs)

    decay_steps, warmup_steps = calc_train_steps(
        len_train,
        batch_size=BATCH_SIZE,
        epochs=NUM_EPOCHS,
    )

    model.compile(
        loss='binary_crossentropy',
        # optimizer=Adam(1e-5),
        optimizer=AdamWarmup(
            decay_steps=decay_steps,
            warmup_steps=warmup_steps,
            lr=LR,
            min_lr=MIN_LR,
            ))
    del bert_model
    gc.collect()
    return model


## train on answer

In [0]:
test_x = convert_data(test, branch='testing')
pred = np.zeros((len(test), NUM_CLASSES))
kf = KFold(n_splits=5, shuffle=True, random_state=0)
idx = [x for x in kf.split(train)]

def compute_spearmanr(trues, preds):
    rhos = []
    for col_trues, col_pred in zip(trues.T, preds.T):
        rhos.append(spearmanr(col_trues, col_pred + np.random.normal(0, 1e-7, col_pred.shape[0])).correlation)
    return np.nanmean(rhos)

class IntervalEval(Callback):
    def __init__(self, test_data, label):
        super(Callback, self).__init__()
        self.test_data = test_data
        self.label = label

    def on_epoch_end(self, epoch, logs={}):
        # if epoch == 3:
        preds = self.model.predict_generator(self.test_data.__iter__(), len(self.test_data), verbose=1)
        score = compute_spearmanr(self.label, preds)
        print('Spearman - {:.5f}'.format(score))

for i, (tr_idx, val_idx) in enumerate(idx[0:1], 1):
    print('\nFold - {:}\n'.format(i))
    tr, val = train.loc[tr_idx], train.loc[val_idx]
    tr_x, tr_y = convert_data(tr)
    val_x, val_y = convert_data(val)
    
    model = model_build(len_train=len(tr))

    train_D = data_generator(list(zip(tr_x[0], tr_x[1], tr_x[2], tr_y)))
    valid_D = data_generator(list(zip(val_x[0], val_x[1], val_x[2], val_y)), branch='valid')
    test_D = data_generator(list(zip(test_x[0], test_x[1], test_x[2])), branch='test')
    ieval = IntervalEval(test_data=valid_D, label=val_y)
    model.fit_generator(
        train_D.__iter__(),
        steps_per_epoch=len(train_D),
        epochs=NUM_EPOCHS,
        callbacks = [ieval]
    )
    # oof_pred = model.predict_generator(valid_D.__iter__(), len(valid_D), verbose=1)
    # train_aug.loc[val_idx, 'oof_pred'] = np.argmax(oof_pred, 1)
    # print('oof - {:} f1_score - {:.4f}'.format(i, spearmanr(val_y, np.argmax(oof_pred, 1), average='weighted')))

    pred+= model.predict_generator(test_D.__iter__(), len(test_D), verbose=1) / kf.get_n_splits()
    # np.save(OUTPUT_TEST, pred)
    # s3.upload_file(Filename=OUTPUT_TEST, Bucket='acmilannesta', Key='large/'+OUTPUT_TEST)

    # model_file = 'model-oof-'+str(i)+'.h5'
    # model.save('model.h5')

    del model
    gc.collect()

Epoch 1/3


In [16]:
BATCH_SIZE

4